在 AKS 或 Arc Kubernetes 群集上部署 Azure 机器学习扩展后，可以将 Kubernetes 群集附加到 Azure 机器学习工作区，并创建要供 ML 专业人员使用的计算目标。

## 前置条件

将 Kubernetes 群集附加到 Azure 机器学习工作区可以灵活地支持许多不同的方案。 例如，具有多个附件的共享方案、访问 Azure 资源的模型训练脚本以及工作区的身份验证配置。

In [ ]:
cluster_name="<AKS集群名>"
resource_group="<资源组名>"
workspace="<工作区名>"
subscribe_id ="<订阅id>"

resource_id 是唯一标识K8s集群的资源id，一般格式为：

In [ ]:
resource_id=f"/subscriptions/{subscribe_id}/resourceGroups/{resource_group}/providers/Microsoft.ContainerService/managedclusters/{cluster_name}"

执行下面这个命令用于将 Kubernetes 计算资源附加到 Azure Machine Learning 工作区：

```bash
az ml compute attach \
    --resource-group {resource_group} \     # 资源组名称
    --workspace-name {workspace} \          # ML 工作区名称
    --type Kubernetes \                     # 计算资源类型
    --name k8s-compute \                    # 计算资源名称
    --resource-id resource-id \                         # K8s 集群资源 ID
    --identity-type SystemAssigned \        # 身份类型
    --namespace {aks_namespace} \           # K8s 命名空间
    --no-wait                               # 异步执行
```

详细参数说明：

1. **基本标识参数**：
- `--resource-group {resource_group}`:
  - 指定 Azure Machine Learning 工作区所在的资源组
  - 必须是已存在的资源组

- `--workspace-name {workspace}`:
  - Azure Machine Learning 工作区的名称
  - 必须是已创建的工作区

2. **计算资源配置**：
- `--type Kubernetes`:
  - 指定要附加的计算资源类型
  - 这里指定为 Kubernetes，表示要附加 K8s 集群

- `--name k8s-compute`:
  - 为这个计算资源指定一个名称
  - 在工作区中必须是唯一的
  - 这个名称将用于在 ML 工作区中引用此计算资源

3. **Kubernetes 相关参数**：
- `--resource-id`:
  - Kubernetes 集群的 Azure 资源 ID
  - 用于唯一标识要附加的 K8s 集群
  - 格式通常为：/subscriptions/{subscription-id}/resourceGroups/{resource-group}/providers/Microsoft.ContainerService/managedClusters/{cluster-name}

- `--namespace {aks_namespace}`:
  - 指定在 Kubernetes 集群中使用的命名空间
  - 用于隔离 ML 工作负载
  - 如果命名空间不存在，将会自动创建

4. **身份验证参数**：
- `--identity-type SystemAssigned`:
  - 指定使用的身份类型, 可选 `SystemAssigned` 货 UserAssigned 托管标识
  - `SystemAssigned` 表示使用系统分配的托管标识
  - 仅 `--user-assigned-identities` 托管标识需要 `UserAssigned`。 虽然可以提供以逗号分隔的用户托管的标识列表，但在附加群集时仅使用第一个标识。
  - 这将允许 ML 工作区使用托管标识访问 K8s 集群

5. **执行控制参数**：
- `--no-wait`:
  - 指示命令立即返回，不等待操作完成
  - 适用于长时间运行的操作
  - 可以使用其他命令查询操作状态


计算附加不会自动创建 Kubernetes 命名空间，也不会验证 kubernetes 命名空间是否存在。 你需要验证群集中是否存在指定的命名空间，否则提交到此计算的任何 Azure 机器学习工作负载都将失败。

## 将托管标识分配给计算目标

开发人员面临的一个共同挑战是如何管理密码和凭据，以确保解决方案的不同组件之间的通信安全。 [托管标识](https://learn.microsoft.com/zh-cn/azure/active-directory/managed-identities-azure-resources/overview)使开发人员无需管理凭据。

若要访问用于 Docker 映像的 Azure 容器注册表 (ACR)，以及用于训练数据的存储帐户，请在启用系统分配或用户分配的托管标识的情况下附加 Kubernetes 计算。

### 分配托管标识

- 可以在计算附加步骤中将托管标识分配给计算。
    
- 如果已附加计算，则可以更新设置以在 Azure 机器学习工作室中使用托管标识。
    
    - 转到 [Azure 机器学习工作室](https://ml.azure.com/)。 选择“计算”、“附加的计算”，然后选择附加的计算。
    - 选择铅笔图标以编辑托管标识。

    
    ![从 Azure 门户更新 Kubernetes 计算标识的屏幕截图。](https://learn.microsoft.com/zh-cn/azure/machine-learning/media/how-to-attach-kubernetes-to-workspace/edit-identity.png?view=azureml-api-2)
    
    ![从 Azure 门户选择 Kubernetes 计算标识的屏幕截图。](https://learn.microsoft.com/zh-cn/azure/machine-learning/media/how-to-attach-kubernetes-to-workspace/update-identity-2.png?view=azureml-api-2)
    

### 向托管标识分配 Azure 角色

Azure 提供了几种向托管标识分配角色的方法。

- [使用 Azure 门户分配角色](https://learn.microsoft.com/zh-cn/azure/role-based-access-control/role-assignments-portal)
- [使用 Azure CLI 分配角色](https://learn.microsoft.com/zh-cn/azure/role-based-access-control/role-assignments-cli)
- [使用 Azure PowerShell 分配角色](https://learn.microsoft.com/zh-cn/azure/role-based-access-control/role-assignments-powershell)

如果使用 Azure 门户分配角色并具有**系统分配的托管标识**，**请选择“用户”**、“**组主体**”或“**服务主体**”，可以通过选择“**选择成员**”来搜索标识名称。 标识名称的格式需要设置为 `<workspace name>/computes/<compute target name>`。

如果具有用户分配的托管标识，请选择“托管标识”以查找目标标识。

可使用托管标识从 Azure 容器注册表拉取映像。 向计算托管标识授予 AcrPull 角色。 有关详细信息，请参阅 [Azure 容器注册表角色和权限](https://learn.microsoft.com/zh-cn/azure/container-registry/container-registry-roles)。

可使用托管标识访问 Azure Blob：

- 出于只读目的，应向计算托管标识授予“存储 Blob 数据读取者”角色。
- 出于读写目的，应向计算托管标识授予“存储 Blob 数据参与者”角色。